In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
# ---- Import Required Libraries ----
import pathway as pw
import panel as pn
import bokeh.plotting
import datetime


In [3]:
df=pd.read_csv("/content/dataset.csv")

In [4]:
df.head()
print('Data loaded successfully. Shape of the dataframe:', df.shape)

Data loaded successfully. Shape of the dataframe: (18368, 12)


In [5]:
df.isnull().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [6]:
df.drop_duplicates()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [7]:
df=df.drop("ID",axis=1)

In [8]:
vehicle_type_dummies = pd.get_dummies(df["VehicleType"], dtype=int,prefix="VehicleType")
df = pd.concat([df, vehicle_type_dummies], axis=1)
df = df.drop("VehicleType", axis=1)

In [9]:
traffic_condition=pd.get_dummies(df['TrafficConditionNearby'],dtype=int,prefix='TrafficCondition')
df=pd.concat([df,traffic_condition],axis=1)
df=df.drop('TrafficConditionNearby',axis=1)

In [10]:
df["LastUpdatedDate"]=pd.to_datetime(df["LastUpdatedDate"], format="%d-%m-%Y")

In [11]:
df["LastUpdatedTime"]=pd.to_datetime(df["LastUpdatedTime"], format="%H:%M:%S")

In [12]:
df["occupancy_pct"] = (df["Occupancy"] / df["Capacity"]) * 100

In [13]:
df['hour_of_day'] = df['LastUpdatedTime'].dt.hour

In [14]:
df['day_of_week'] = df['LastUpdatedDate'].dt.dayofweek
df['weekend_flag']=df['day_of_week'].apply(lambda x: 1 if x in [5,6] else 0)

In [15]:
df.head()

,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,VehicleType_bike,VehicleType_car,VehicleType_cycle,VehicleType_truck,TrafficCondition_average,TrafficCondition_high,TrafficCondition_low,occupancy_pct,hour_of_day,day_of_week,weekend_flag
0,BHMBCCMKT01,577,26.144536,91.736172,61,1,0,2016-10-04,1900-01-01 07:59:00,0,1,0,0,0,0,1,10.571924,7,1,0
1,BHMBCCMKT01,577,26.144536,91.736172,64,1,0,2016-10-04,1900-01-01 08:25:00,0,1,0,0,0,0,1,11.091854,8,1,0
2,BHMBCCMKT01,577,26.144536,91.736172,80,2,0,2016-10-04,1900-01-01 08:59:00,0,1,0,0,0,0,1,13.864818,8,1,0
3,BHMBCCMKT01,577,26.144536,91.736172,107,2,0,2016-10-04,1900-01-01 09:32:00,0,1,0,0,0,0,1,18.544194,9,1,0
4,BHMBCCMKT01,577,26.144536,91.736172,150,2,0,2016-10-04,1900-01-01 09:59:00,1,0,0,0,0,0,1,25.996534,9,1,0


In [16]:
df['Timestamp'] = df['LastUpdatedDate'] + pd.to_timedelta(df['LastUpdatedTime'].dt.strftime('%H:%M:%S'))

In [17]:
df = df.sort_values(by='Timestamp')

In [18]:
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

In [19]:
class ParkingSchema(pw.Schema):
    Timestamp: pw.DateTimeNaive  # or pw.DateTimeUtc, depending on your data
    Capacity: int
    Occupancy: int

# Now, when reading:
data = pw.io.csv.read(
    "parking_stream.csv",
    schema=ParkingSchema,
    # Optionally, specify your datetime format
    # But Pathway will often autodetect: if not, see below
)

In [20]:
# Linear price based on occupancy
# price_t = price_t-1 + alpha * (occupancy / capacity)
# Start with base price $10
alpha = 5  # Example value

def compute_price(occupancy, capacity, prev_price=10):
    return prev_price + alpha * (occupancy / capacity)

In [21]:
data_with_time = data.with_columns(
    t=pw.this.Timestamp,
    day=pw.apply(lambda ts: ts.date(), pw.this.Timestamp)
)
# Now proceed with windowing using the new 't' and 'day'
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)
# ---- Panel and Bokeh setup ----
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

# ---- Stream + plot ----
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

# **Model 2**

In [22]:
class ParkingSchema2(pw.Schema):
    Timestamp: pw.DateTimeNaive  # native datetime
    Capacity: int
    Occupancy: int
    QueueLength: int
    TrafficConditionNearby: int
    IsSpecialDay: int
    VehicleType: str

In [23]:
data = pw.io.csv.read(
    "parking_stream2.csv",  # <-- your file name here
    schema=ParkingSchema2,
)

In [24]:
data_with_time = data.with_columns(
    t=pw.this.Timestamp,
    day=pw.apply(lambda ts: ts.date(), pw.this.Timestamp)
)

In [27]:
# --- Step 3: Add demand column as float ---
def vehicle_type_weight(vtype: str) -> float:
    if vtype is None:
        return 1.0
    vtype = vtype.lower()
    if vtype == "car":
        return 1.0
    elif vtype == "bike":
        return 0.7
    elif vtype == "truck":
        return 1.3
    else:
        return 1.0

def compute_demand(occ, cap, queue, traffic, special, vtype):
    α, β, γ, δ, ε = 1.0, 1.0, 1.0, 1.0, 1.0
    vt_wt = vehicle_type_weight(vtype)
    value = (α * (occ / cap)) + (β * queue) - (γ * traffic) + (δ * special) + (ε * vt_wt)
    return float(value)

data_with_demand = data_with_time.with_columns(
    demand=pw.apply(
        compute_demand,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.VehicleType
    )
)

# --- Step 4: Aggregate min/max demand ---
agg = data_with_demand.reduce(
    min_demand=pw.reducers.min(pw.this.demand),
    max_demand=pw.reducers.max(pw.this.demand),
)
data_with_norm = data_with_demand.join(agg)

# --- Step 5: Calculate price (always work on float) ---
def get_price(base, demand, min_d, max_d, lam=0.5):
    # All math uses float and all return statements must be float
    try:
        if max_d - min_d == 0:
            norm = 0.5
        else:
            norm = (demand - min_d) / (max_d - min_d)
        price = base * (1 + lam * norm)
        price = min(max(price, 0.5 * base), 2 * base)
        return float(round(price, 2))
    except Exception:
        return float(base)  # Always return a float in error case

final_data = data_with_norm.select(
    **{col: pw.this[col] for col in data_with_demand.schema.columns()},
    min_demand=pw.this.min_demand,
    max_demand=pw.this.max_demand,
).with_columns(
    price=pw.apply(get_price, 10.0, pw.this.demand, pw.this.min_demand, pw.this.max_demand)
)
# Add a new column that forcibly casts price to float
# Suppose you have a column "price" that you built with pw.apply and it’s a float
final_data = data_with_demand.with_columns(
    price=pw.apply(get_price, 10.0, pw.this.demand, pw.this.demand, pw.this.demand)
)
# The above just for testing—get_price is run with all demands, so min_demand == max_demand == demand
# See if reducers now work:
delta_window = (
    final_data.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        price_sum=pw.reducers.sum(pw.this.price),
        price_count=pw.reducers.count(pw.this.price),
        price_min=pw.reducers.min(pw.this.price),
        price_max=pw.reducers.max(pw.this.price),
    )
    .with_columns(
        price_avg=pw.this.price_sum / pw.this.price_count
    )
)

# --- Step 7: Visualization ---
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Demand-Based Dynamic Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price_avg", source=source, line_width=2, color="navy", legend_label="Avg Price")
    fig.circle("t", "price_avg", source=source, size=5, color="red")
    fig.line("t", "price_min", source=source, line_width=1, color="green", legend_label="Min Price", line_dash='dashed')
    fig.line("t", "price_max", source=source, line_width=1, color="orange", legend_label="Max Price", line_dash='dashed')
    fig.legend.location = "top_left"
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

TypeError: Pathway does not support using reducer sum on column of type ANY.


In [28]:
# Use your schema, where Capacity is int
test_table = data_with_time.with_columns(
    test_col=pw.this.Capacity  # This is guaranteed to be int
)
agg = test_table.reduce(
    test_sum=pw.reducers.sum(pw.this.test_col)
)
print(test_table.schema.columns())
print(test_table.take(1)[0])

{'Timestamp': ColumnSchema(dtype=DATE_TIME_NAIVE, name='Timestamp', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'Capacity': ColumnSchema(dtype=INT, name='Capacity', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'Occupancy': ColumnSchema(dtype=INT, name='Occupancy', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'QueueLength': ColumnSchema(dtype=INT, name='QueueLength', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'TrafficConditionNearby': ColumnSchema(dtype=INT, name='TrafficConditionNearby', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'IsSpecialDay': ColumnSchema(dtype=INT, name='IsSpecialDay', primary_key=False, default_value=undefined, append_only=False, description=None, example=None), 'VehicleType': ColumnSchema(dtype=STR, name='Ve

AttributeError: Table has no column with name take.